# Sentinel Zonal Stats through Google Earth Engine

## A. Python start

In [1]:
#Inladen van de noodzakelijke packages
import ee
import geemap
import geetools
from ipygee import *
import numpy as np
import pandas as pd
import geopandas as gp
import folium
import math
import matplotlib as plt

# File with custom created functions:
from GEE_SEN import *

C:\ProgramData\Anaconda3\envs\phd_01\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
#Indien nodig: Authorizeren in Earth Engine
ee.Authenticate()

Enter verification code:  4/1AX4XfWiLEA_nEPPOA-mlREE1PPARmQRmV9-QGtIsPIKqLrX1CmiGaCv5SLw



Successfully saved authorization token.


In [3]:
#Earth Engine initialiseren
ee.Initialize()

# A. GEDI data

In [4]:
#Load GEDI-shot data to pandas
Gedi_2A = gp.read_file(r'F:\Users\jfeyen\PhD\Python\04_extracted_GEDI\gediDF_2A_Flanders_26092021_polygons\gediDF_2A_Flanders_26092021_polygons.shp')
Gedi_2B = gp.read_file(r'F:\Users\jfeyen\PhD\Python\04_extracted_GEDI\gediDF_2B_Flanders_26092021_polygons\gediDF_2B_Flanders_26092021_polygons.shp')

In [ ]:
# Make a random selection in GEDI data to limit the train/test size


## B. Region of Interest: studiegebied definiëren

In [11]:
ROI_file = r'F:\Users\jfeyen\PhD\Python\01_Data_exploration\Data\VL_shape.zip'
ROI= gp.read_file(ROI_file)
BOS_file = r'F:\Users\jfeyen\PhD\DATA\Vectordata\Bos\Bosreferentielaag\BosreferentielaagVL.shp'
BOS= gp.read_file(BOS_file).to_crs(epsg='31370')

### C. Filtering of the GEDI SHOTS 


In [8]:
## Example: select MA-SEPT 2019
START_DATE = '2019-03-15'
END_DATE = '2019-10-15'
outputfile2A = r'F:\Users\jfeyen\PhD\Python\04_extracted_GEDI\GEDI2A_15032019to15102019_BOS'
outputfile2B = r'F:\Users\jfeyen\PhD\Python\04_extracted_GEDI\GEDI2B_15032019to15102019_BOS'

In [9]:
def selectExportGEDIbyDate(Gedi,start_date,end_date,outputfile):
    import datetime
    def getDateTimeGEDI(fileLoc):
        juliandate = fileLoc[11:16]
        return datetime.datetime.strptime(juliandate, '%y%j')#.strftime("%Y-%m-%d")
    
    Gedi['Date'] = Gedi['fileLoc'].apply(getDateTimeGEDI)
    GEDI_sel= Gedi[((Gedi['Date']> START_DATE) & (Gedi['Date'] < END_DATE))]
    GEDI_sel['Date']=GEDI_sel['Date'].astype(str)
    
    GEDI_sel.to_file(outputfile)
    return print('Saved as ' + outputfile)

In [10]:
Gedi_2A_bos = gp.clip(Gedi_2A, BOS)
Gedi_2B_bos =  gp.clip(Gedi_2B, BOS)

TypeError: 'mask' should be GeoDataFrame, GeoSeries or(Multi)Polygon, got <class 'ee.featurecollection.FeatureCollection'>

In [54]:
selectExportGEDIbyDate(Gedi_2A, START_DATE,END_DATE, outputfile2A)
selectExportGEDIbyDate(Gedi_2B, START_DATE,END_DATE, outputfile2B)

C:\ProgramData\Anaconda3\envs\phd_01\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


Saved as F:\Users\jfeyen\PhD\Python\04_extracted_GEDI\GEDI2A_15032019to15102019_BOS
Saved as F:\Users\jfeyen\PhD\Python\04_extracted_GEDI\GEDI2B_15032019to15102019_BOS


In [6]:
Map = geemap.Map()
Map.addLayer(BOS)
Map.centerObject(ROI,9)

# Earth Engine Code

In [13]:
#Linkjes leggen naar ROI's in Earth Engine assets
ROI = ee.FeatureCollection("users/jasperfeyen/UGENT/Vlaanderen_shape")
BOS = ee.FeatureCollection("users/jasperfeyen/UGENT/BosreferentielaagVL")

## CLOUD MASK FUNCTIONS

In [14]:
# Functie aanmaken die S2_SR collectie koppelt aan de s2cloudless info
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


In [15]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [16]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [17]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img.addBands(is_cld_shdw)

In [18]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

## C. Function to Create Sentinel image

In [19]:
def createSentinelImage(ROI, S2_bands,TextureMetrics, START_DATE, END_DATE, window_size):
    # Step 1: create Cloud Masked s2 collection
    s2_sr_cld_col = get_s2_sr_cld_col(ROI, START_DATE, END_DATE)
    S2_2019_coll = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)
    
    #Step2: Add Spectral Indices
    S2_2019 = S2_2019_coll.map(addIndices).select(bands).median().clip(ROI)
    
    #Step3: Add S2 Textural Indices
    S2_2019 = addS2Texturebands(S2_2019, window_size,TextureMetrics)
    
    #Step4: Add S1 (Ascending)
    S1_asc = refinedLee(add_S1_asc(createS1coll(ROI), START_DATE, END_DATE))
    
    #Step 5: add Texture to S1
    S1 = addS1Texture(S1_asc, TextureMetrics, window_size)
    
    #Step 6: Combine S1/S2
    S1S2 = S2_2019.addBands(S1).clip(ROI)

    return S1S2
    
    

In [20]:
ROI = ee.FeatureCollection("users/jasperfeyen/UGENT/Vlaanderen_shape")
BOS = ee.FeatureCollection("users/jasperfeyen/UGENT/BosreferentielaagVL")

In [21]:
AOI = ROI.geometry();
#TEST: zomer
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

In [22]:
# Spectral features
S2_bands = ['B2','B3', 'B4', 'B5', 'B6', 'B7','B8','B8A','B11','B12']
# Vegetation Indices
VI = ['NDVI','MNDWI','NDWI','IRECI','SAVI','S2REP']

bands = S2_bands + VI

# Texture: https://developers.google.com/earth-engine/apidocs/ee-image-glcmtexture
TextureMetrics = ['asm','contrast','corr','var','savg','idm','diss','ent']

In [23]:
#Start by defining window size: experiment with 3x3 - 5x5 - 7x7 - 9x9
window_size = 5

In [24]:
S1S2_2019 = createSentinelImage(ROI, bands,TextureMetrics, START_DATE, END_DATE, window_size)

In [25]:
import geopandas as gp
from shapely.geometry import Point
from shapely.wkt import loads

## NEXT STEP: UPLOAD THE DATA TO EARTH ENGINE (NEEDS AUTOMATISATION: TO DO)


ReduceRegions neemt veel tijd in beslag => mapping ReduceRegion zou beter zijn
https://gis.stackexchange.com/questions/346052/exporting-dataframe-used-reduceregion-to-sample-an-image-in-google-earth-engin

In [38]:
#Define the table in GEE
#START_DATE = '2019-10-15'
#END_DATE = '2020-03-15'
GEDI_2B= ee.FeatureCollection('users/jasperfeyen/GEE_EXTRACTS/GEDI2A_15032019to15102019_BOS')

In [27]:
# Sentinel collections
S1S2_SUMMER2019 = createSentinelImage(ROI.geometry(), bands,TextureMetrics, START_DATE, END_DATE, window_size)

In [28]:
def SampleS1S2_2_feature(S1S2):
    def wrap(feature):
        return feature.setMulti(S1S2.clip(ROI).float().reduceRegion(
            reducer = ee.Reducer.mean(),
            geometry= feature.geometry(),
            scale = 10,
            bestEffort = True,
            tileScale = 16
  ))
    return wrap


### Take a random sample of the shots -> Limiting processing time

In [53]:
#Make a random Column
GEDI_2B_random = GEDI_2B.randomColumn()
GEDI_2B_selected= GEDI_2B_random.filter(ee.Filter.gt('random',0.95));

In [54]:
print(GEDI_2B_selected.size().getInfo(), 'selected from ', GEDI_2B.size().getInfo())

6175 selected from  120619


In [55]:
GEDI_S1S2 = GEDI_2B_selected.map(SampleS1S2_2_feature(S1S2_SUMMER2019));

In [56]:
GEDI_S1S2.first().propertyNames().getInfo()

['B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B11',
 'B12',
 'NDVI',
 'MNDWI',
 'NDWI',
 'IRECI',
 'SAVI',
 'S2REP',
 'B2_asm',
 'B3_asm',
 'B4_asm',
 'B5_asm',
 'B6_asm',
 'B7_asm',
 'B8_asm',
 'B8A_asm',
 'B11_asm',
 'B12_asm',
 'NDVI_asm',
 'MNDWI_asm',
 'NDWI_asm',
 'IRECI_asm',
 'SAVI_asm',
 'S2REP_asm',
 'B2_contrast',
 'B3_contrast',
 'B4_contrast',
 'B5_contrast',
 'B6_contrast',
 'B7_contrast',
 'B8_contrast',
 'B8A_contrast',
 'B11_contrast',
 'B12_contrast',
 'NDVI_contrast',
 'MNDWI_contrast',
 'NDWI_contrast',
 'IRECI_contrast',
 'SAVI_contrast',
 'S2REP_contrast',
 'B2_corr',
 'B3_corr',
 'B4_corr',
 'B5_corr',
 'B6_corr',
 'B7_corr',
 'B8_corr',
 'B8A_corr',
 'B11_corr',
 'B12_corr',
 'NDVI_corr',
 'MNDWI_corr',
 'NDWI_corr',
 'IRECI_corr',
 'SAVI_corr',
 'S2REP_corr',
 'B2_var',
 'B3_var',
 'B4_var',
 'B5_var',
 'B6_var',
 'B7_var',
 'B8_var',
 'B8A_var',
 'B11_var',
 'B12_var',
 'NDVI_var',
 'MNDWI_var',
 'NDWI_var',
 'IRECI_var',
 'SAVI_var',
 'S2REP_var'

In [57]:
task = ee.batch.Export.table.toDrive(
    collection = GEDI_S1S2,
    folder= 'GEE_EXPORTS',
    fileNamePrefix = ' GEDIS1S2_15032019to15102019_BOS_18112021',
    description='GEDIS1S2_15032019to15102019_BOS_18112021',
    fileFormat = 'SHP'
)

In [58]:
task.start()

In [87]:
task.status()["state"]

'RUNNING'

### Level 2B

In [215]:
GediDF = ee.FeatureCollection('users/jasperfeyen/GEDI_EXTRACTS/GEDI_2B_15032019to15092019')

In [216]:
GediDF.first().propertyNames().getInfo()

['Sensitivit',
 'Plant Area',
 'Degrade Fl',
 'Canopy Hei',
 'index',
 'L2B Qualit',
 'Foliage He',
 'Shot Numbe',
 'fileLoc',
 'Date',
 'Beam',
 'cover',
 'Elevation',
 'Pgap_theta',
 'Canopy Ele',
 'PAI',
 'Tandem-X D',
 'system:index',
 'Solar_elev']

In [72]:
# Start Sampling over collection
# Bands for training
GEDI_S1S2 = S1S2_2019.clip(ROI).float().reduceRegions(
    reducer =  ee.Reducer.mean(),
    collection = GediDF,
    scale = 10, #Sentinel-2 maximum scale
    tileScale = 16)

ReduceRegions neemt veel tijd in beslag => mapping ReduceRegion zou beter zijn
https://gis.stackexchange.com/questions/346052/exporting-dataframe-used-reduceregion-to-sample-an-image-in-google-earth-engin

In [73]:
GEDI_S1S2 = S1S2_2019.clip(ROI).float().reduceRegion(
    reducer =  ee.Reducer.mean(),
    geometry = GediDF.first().geometry(),
    scale = 10, #Sentinel-2 maximum scale,
    bestEffort = True,
    tileScale = 16)
test=GediDF.first().setMulti(GEDI_S1S2)

In [74]:
GEDI_S1S2 = GediDF.first().set(S1S2_2019.clip(ROI).float().reduceRegion(
    reducer= ee.Reducer.mean(),
    geometry= GediDF.first().geometry(),
    scale=10
  ))

In [75]:
def SampleS1S2_2_feature(feature):
    return feature.setMulti(S1S2_2019.clip(ROI).float().reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry= feature.geometry(),
        scale = 10
  ))

In [76]:
GEDI_S1S2 = GediDF.map(SampleS1S2_2_feature);

In [77]:
GEDI_S1S2.first().propertyNames().getInfo()

['B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B11',
 'B12',
 'NDVI',
 'MNDWI',
 'NDWI',
 'IRECI',
 'SAVI',
 'S2REP',
 'B2_asm',
 'B3_asm',
 'B4_asm',
 'B5_asm',
 'B6_asm',
 'B7_asm',
 'B8_asm',
 'B8A_asm',
 'B11_asm',
 'B12_asm',
 'NDVI_asm',
 'MNDWI_asm',
 'NDWI_asm',
 'IRECI_asm',
 'SAVI_asm',
 'S2REP_asm',
 'B2_contrast',
 'B3_contrast',
 'B4_contrast',
 'B5_contrast',
 'B6_contrast',
 'B7_contrast',
 'B8_contrast',
 'B8A_contrast',
 'B11_contrast',
 'B12_contrast',
 'NDVI_contrast',
 'MNDWI_contrast',
 'NDWI_contrast',
 'IRECI_contrast',
 'SAVI_contrast',
 'S2REP_contrast',
 'B2_corr',
 'B3_corr',
 'B4_corr',
 'B5_corr',
 'B6_corr',
 'B7_corr',
 'B8_corr',
 'B8A_corr',
 'B11_corr',
 'B12_corr',
 'NDVI_corr',
 'MNDWI_corr',
 'NDWI_corr',
 'IRECI_corr',
 'SAVI_corr',
 'S2REP_corr',
 'B2_var',
 'B3_var',
 'B4_var',
 'B5_var',
 'B6_var',
 'B7_var',
 'B8_var',
 'B8A_var',
 'B11_var',
 'B12_var',
 'NDVI_var',
 'MNDWI_var',
 'NDWI_var',
 'IRECI_var',
 'SAVI_var',
 'S2REP_var'

In [78]:
task = ee.batch.Export.table.toDrive(
    collection = GEDI_S1S2,
    folder= 'GEE_EXPORTS',
    fileNamePrefix = 'GEDI_2B_S1S2_WINTER_5X5',
    description='GEDI_S1S2_Flanders_13092021',
)

In [79]:
task.start()

In [62]:
task.status()

{'state': 'RUNNING',
 'description': 'GEDI_S1S2_Flanders_13092021',
 'creation_timestamp_ms': 1632664462889,
 'update_timestamp_ms': 1632838085785,
 'start_timestamp_ms': 1632701435400,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 2,
 'id': 'Z5O3JIV6WCEI2WR4AHDR2MYZ',
 'name': 'projects/earthengine-legacy/operations/Z5O3JIV6WCEI2WR4AHDR2MYZ'}

## HERE: try to find a way to automatically upload tables to GEE

In [36]:
# convert it into geo-json 
import ee
import geemap
import json
import os
json_df = json.loads(GEDI_SUMMER2019.drop(columns="Date").to_json())
    
# create a gee object with geemap
ee_object = geemap.geojson_to_ee(json_df)
    
# upload this object to earthengine
asset = 'users/jasperfeyen/UGENT/Gedi_2B_Summer2019'
            
#create and launch the task
task_config = {
    'collection': ee_object, 
    'description':'Gedi_2B_Summer2019',
    'assetId': asset
}
task = ee.batch.Export.table.toAsset(**task_config)
task.start()

MemoryError: 

In [45]:
taskID = ee.data.newTaskId()
params = {
    'id': 'users/jasperfeyen/UGENT/GEDI_SUMMER2019', 
    'description':'Gedi_2B_Summer2019',
    'assetId': taskID
}
ee.data.startTableIngestion(taskID, params, allow_overwrite=False)

EEException: Invalid JSON payload received. Unknown name "description" at 'table_manifest': Cannot find field.
Invalid JSON payload received. Unknown name "assetId" at 'table_manifest': Cannot find field.
Invalid JSON payload received. Unknown name "requestId": Proto field is not repeating, cannot start list.